In [1]:
import pandas as pd
import numpy as np
import ast
import re
import os

from atoss.process import *

In [2]:
def count_O(group):
    return (group == 'O').sum()
def count_Os(group):
    return (group == 'OOOO').sum()

In [4]:
# compute f1 score by task, dataset, type(quad/item) 
method = 'mvp'
llm = 'zero'

data_types = ['train', 'dev']
f1_types = ['quad', 'item']
task_datasets = [['asqp','rest15'],['asqp','rest16'],['acos','rest16'],['acos','laptop16']]
data_counts =  [[834, 606, 723, 2934],[209, 277, 133, 326]]
home = os.path.expanduser('~/')


for data_type in data_types:
    for f1_type in f1_types:
        fi_score = pd.DataFrame()
        for task_dataset in task_datasets:
            task, dataset = task_dataset[0], task_dataset[1]

            if method == 'mvp':
                output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/top_5_post_data1.0')
            elif method == 'dlo':
                output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/top_5_post_data1.0')
            elif method == 'paraphrase':
                output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/post_data1.0')
            
            for i in range(0, 11):
        
                file_name = f'{task}_{dataset}_{data_type}_{llm}_{i}.xlsx'
                file_path = os.path.join(output_dir, file_name)
                df = pd.read_excel(file_path)
                if f1_type == 'quad':
                    df = df.groupby(['sent_id', 'sentence']).agg({'max_ord_t': 'max','max_ord_p': 'max',
                                                                  'score': count_Os}).reset_index()
                    df['score'] = df['score'].fillna(0)
                    df['precision'] = df['score']/(df['max_ord_p'])
                    df['recall'] = df['score']/(df['max_ord_t'])
                elif f1_type == 'item':
                    df = df.groupby(['sent_id', 'sentence']).agg({'max_ord_t': 'max','max_ord_p': 'max',
                                                                  'A_err': count_O,'C_err': count_O,'O_err':count_O,'S_err':count_O}).reset_index()   
                    df['score'] = df['A_err'] + df['C_err'] + df['O_err']+ df['S_err']
                    df['score'] = df['score'].fillna(0)
                    df['precision'] = df['score']/(df['max_ord_p']*4)
                    df['recall'] = df['score']/(df['max_ord_t']*4)
                df['f1'] = 2 * df['precision'] * df['recall'] / (df['precision'] + df['recall'])
                df['f1'] = df['f1'].fillna(0)
                df['f1'] = round(df['f1'] * 100, 2)
                df.rename(columns={'f1': f'f1_{i}'}, inplace=True)
                df.rename(columns={'sentence': f'sent_{i}'}, inplace=True)
                if i == 0:
                    f1_df = df[['sent_id','max_ord_t',f'sent_{i}',f'f1_{i}']]
                else:
                    f1_df[f'sent_{i}'] = df[f'sent_{i}']
                    f1_df[f'f1_{i}'] = df[f'f1_{i}']
        
            f1_df['model'] = f'{task}_{dataset}'
            
            f1_df['dataset'] = ''
            start_index = 0
            
            if data_type == 'train':
                data_count = data_counts[0]
            elif data_type == 'dev':
                data_count = data_counts[1]
            
            for count, name in zip(data_count, task_datasets):
                task1, dataset1 = name[0], name[1]
                if task!=task1 or dataset!=dataset1:
                    end_index = start_index + count
                    f1_df.loc[start_index:end_index-1, 'dataset'] = f'{task1}_{dataset1}'
                    f1_df.loc[start_index:end_index-1, 'data_id'] = range(1, count + 1)
                    start_index = end_index
        
            col_names = list(f1_df.columns)
            f1_col = sorted([col for col in col_names if col.startswith('f1_')], key=lambda x: int(x.split('_')[1]))
            sent_col = sorted([col for col in col_names if col.startswith('sent_') and col != 'sent_id'], key=lambda x: int(x.split('_')[1]))
            sorted_col = ['model','dataset','data_id','sent_id', 'max_ord_t'] + f1_col + sent_col
            
            f1_df = f1_df[sorted_col]
            fi_score = pd.concat([fi_score, f1_df], ignore_index=True)
            print(f'{task}_{dataset} : {len(f1_df)} data add. Total number: {len(fi_score)}')
        
        save_path = os.path.join(home, f'ABSA/outputs/mvp', f'f1_{data_type}_{llm}_{f1_type}.xlsx')
        print(f'finished and save data ==>> {save_path}')
        
        fi_score.to_excel(save_path)

asqp_rest15 : 4263 data add. Total number: 4263
asqp_rest16 : 4491 data add. Total number: 8754
acos_rest16 : 4374 data add. Total number: 13128
acos_laptop16 : 2163 data add. Total number: 15291
finished and save data ==>> /home/elicer/ABSA/outputs/mvp/f1_train_zero_quad.xlsx
asqp_rest15 : 4263 data add. Total number: 4263
asqp_rest16 : 4491 data add. Total number: 8754
acos_rest16 : 4374 data add. Total number: 13128
acos_laptop16 : 2163 data add. Total number: 15291
finished and save data ==>> /home/elicer/ABSA/outputs/mvp/f1_train_zero_item.xlsx
asqp_rest15 : 736 data add. Total number: 736
asqp_rest16 : 668 data add. Total number: 1404
acos_rest16 : 812 data add. Total number: 2216
acos_laptop16 : 619 data add. Total number: 2835
finished and save data ==>> /home/elicer/ABSA/outputs/mvp/f1_dev_zero_quad.xlsx
asqp_rest15 : 736 data add. Total number: 736
asqp_rest16 : 668 data add. Total number: 1404
acos_rest16 : 812 data add. Total number: 2216
acos_laptop16 : 619 data add. Total

In [33]:
# aggregate 10 of data by task, dataset 
method = 'mvp'
llm = 'zero'

data_types = ['train', 'dev']
task_datasets = [['asqp','rest15'],['asqp','rest16'],['acos','rest16'],['acos','laptop16']]
data_counts =  [[834, 606, 723, 2934],[209, 277, 133, 326]]
home = os.path.expanduser('~/')

for data_type in data_types:
    tot_df = pd.DataFrame()
    for task_dataset in task_datasets:
        task, dataset = task_dataset[0], task_dataset[1]
    
        if method == 'mvp':
            output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/top_5_post_data1.0')
        elif method == 'dlo':
            output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/top_5_post_data1.0')
        elif method == 'paraphrase':
            output_dir = os.path.join(home, f'ABSA/outputs/{method}/{task}/{dataset}/post_data1.0')
    
        task_df = pd.DataFrame()
        for i in range(0, 11):
    
            file_name = f'{task}_{dataset}_{data_type}_{llm}_{i}.xlsx'
            file_path = os.path.join(output_dir, file_name)
            df = pd.read_excel(file_path, index_col=False)
            
            df['model'] = f'{task}_{dataset}'
            
            df['dataset'] = ''
            start_index = 1
            
            if data_type == 'train':
                data_count = data_counts[0]
            elif data_type == 'dev':
                data_count = data_counts[1]
            
            for count, name in zip(data_count, task_datasets):
                task1, dataset1 = name[0], name[1]
                if task!=task1 or dataset!=dataset1:
                    end_index = start_index + count - 1
                    df.loc[df['sent_id'].between(start_index, end_index), 'dataset'] = f'{task1}_{dataset1}'
                    sent_ids = df.loc[df['sent_id'].between(start_index, end_index), 'sent_id'].unique()
                    data_id_map = {sid: i+1 for i, sid in enumerate(sorted(sent_ids))}
                    for sid, data_id in data_id_map.items():
                        df.loc[df['sent_id'] == sid, 'data_id'] = data_id
                    start_index = end_index + 1  # 다음 dataset 처리를 위한 start_index 업데이트
    
            df.drop(columns=['Unnamed: 0'], inplace=True)
            cols_to_front = ['model', 'dataset', 'data_id']
            df = df[cols_to_front + [col for col in df.columns if col not in cols_to_front]]
    
            task_df = pd.concat([task_df, df], ignore_index=True)
        tot_df = pd.concat([tot_df, task_df], ignore_index=True)
        print(f'{task}_{dataset} : {len(task_df)} data add. Total number: {len(tot_df)}')
    
    for task_dataset in task_datasets:
        task, dataset = task_dataset[0], task_dataset[1]
    
        save_path = os.path.join(home, f'ABSA/outputs/mvp', f'{task}_{dataset}_{data_type}_{llm}_all.xlsx')
        save_df = tot_df[tot_df['dataset']==f'{task}_{dataset}']
        save_df.to_excel(save_path)
        print(f'finished and save data ==>> {save_path}, Total number : {len(save_df)}')

asqp_rest15 : 75731 data add. Total number: 75731
asqp_rest16 : 78631 data add. Total number: 154362
acos_rest16 : 81131 data add. Total number: 235493
acos_laptop16 : 42751 data add. Total number: 278244
finished and save data ==>> /home/elicer/ABSA/outputs/mvp/asqp_rest15_train_zero_all.xlsx, Total number : 48598
finished and save data ==>> /home/elicer/ABSA/outputs/mvp/asqp_rest16_train_zero_all.xlsx, Total number : 34458
finished and save data ==>> /home/elicer/ABSA/outputs/mvp/acos_rest16_train_zero_all.xlsx, Total number : 43847
finished and save data ==>> /home/elicer/ABSA/outputs/mvp/acos_laptop16_train_zero_all.xlsx, Total number : 151341
asqp_rest15 : 12490 data add. Total number: 12490
asqp_rest16 : 11230 data add. Total number: 23720
acos_rest16 : 15388 data add. Total number: 39108
acos_laptop16 : 12142 data add. Total number: 51250
finished and save data ==>> /home/elicer/ABSA/outputs/mvp/asqp_rest15_dev_zero_all.xlsx, Total number : 12509
finished and save data ==>> /hom